In [19]:
import geopandas as gpd
import pandas as pd
from shapely import box


mgrs = gpd.read_file("../data/mgrs/mgrs_full.fgb")
print(f"Loaded {len(mgrs)} MGRS grid cells.")


Loaded 56686 MGRS grid cells.


In [33]:
Bali = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              115.25786880742186,
              -8.91782452701942
            ],
            [
              115.62994751933928,
              -8.864911327853008
            ],
            [
              115.75679253476551,
              -8.499883644265438
            ],
            [
              115.77088642536938,
              -8.268425611230953
            ],
            [
              115.33961337291942,
              -8.02008471521799
            ],
            [
              115.09719845454964,
              -7.953090213823472
            ],
            [
              114.94498443603698,
              -8.081486649352641
            ],
            [
              114.63210006465204,
              -8.03124939498386
            ],
            [
              114.39250392440204,
              -7.986588838340552
            ],
            [
              114.31921569326715,
              -8.340945335516594
            ],
            [
              114.55881183351715,
              -8.522185531989308
            ],
            [
              114.82941453309331,
              -8.555635923696315
            ],
            [
              114.96471588288063,
              -8.859341073228563
            ],
            [
              115.25786880742186,
              -8.91782452701942
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

In [34]:
#make geojson a geoDataFrame
aoi = gpd.GeoDataFrame.from_features(Bali['features'])
aoi.crs = "EPSG:4326"
mgrs_aoi = mgrs.overlay(aoi)


In [35]:
mgrs_aoi = mgrs_aoi.rename(columns={"Name": "name"})
mgrs_aoi

,name,geometry
0,49LHL,"POLYGON ((114.71716 -8.12502, 114.72125 -8.542..."
1,50LKR,"POLYGON ((115.27352 -8.13836, 115.26998 -8.916..."
2,49MHM,"POLYGON ((114.71798 -8.21330, 114.34500 -8.216..."
3,50MKS,"POLYGON ((115.27314 -8.22679, 114.34389 -8.221..."
4,50MLS,"POLYGON ((115.18438 -8.22640, 115.18544 -7.977..."
5,50LLR,"POLYGON ((115.54638 -8.13915, 115.77089 -8.268..."


In [36]:
print(f"Found {len(mgrs_aoi)} matching MGRS tiles over the AOI.")

Found 6 matching MGRS tiles over the AOI.


In [37]:
mgrs_aoi.to_file("../data/mgrs/mgrs_aoi.fgb")


In [38]:
from ipyleaflet import Map, GeoJSON
# Create a map centered on Bali
m = Map(center=(-8.65, 115.2167), zoom=10)

# Create a GeoJSON layer for the selected MGRS grid cells
geo_json = GeoJSON(data=mgrs_aoi.__geo_interface__, style={"color": "red", "opacity": 0.5, "weight": 1.5, "fillOpacity": 0.5})

# Add the GeoJSON layer to the map
m.add_layer(geo_json)

# Display the map
m



Map(center=[-8.65, 115.2167], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [18]:
```bash
for i in {0..6}; do

python scripts/pipeline/datacube.py \
    --sample data/mgrs/mgrs_aoi.fgb \
    --localpath data/chips  \
    --index $i \
    --dateranges 2023-01-01/2024-12-15 
done
```

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (439736660.py, line 8)

In [42]:
import os
import shutil
import geopandas as gpd
import rasterio
from shapely.geometry import box, shape

# Update the tif_folder path to point directly to the folder
tif_folder = '../data/Bali'
out_folder = os.path.join(tif_folder, 'out')

# GeoJSON data as a dictionary
bali_geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "coordinates": [
                    [
                        [115.25786880742186, -8.91782452701942],
                        [115.62994751933928, -8.864911327853008],
                        [115.75679253476551, -8.499883644265438],
                        [115.77088642536938, -8.268425611230953],
                        [115.33961337291942, -8.02008471521799],
                        [115.09719845454964, -7.953090213823472],
                        [114.94498443603698, -8.081486649352641],
                        [114.63210006465204, -8.03124939498386],
                        [114.39250392440204, -7.986588838340552],
                        [114.31921569326715, -8.340945335516594],
                        [114.55881183351715, -8.522185531989308],
                        [114.82941453309331, -8.555635923696315],
                        [114.96471588288063, -8.859341073228563],
                        [115.25786880742186, -8.91782452701942]
                    ]
                ],
                "type": "Polygon"
            }
        }
    ]
}

# Ensure the output directory exists
os.makedirs(out_folder, exist_ok=True)

# Load the GeoJSON as a GeoDataFrame
bali_polygon = shape(bali_geojson['features'][0]['geometry'])
area_gdf = gpd.GeoDataFrame(index=[0], crs="EPSG:4326", geometry=[bali_polygon])

# Iterate over each TIFF file in the folder
for filename in os.listdir(tif_folder):
    if filename.endswith('.tif'):
        tif_path = os.path.join(tif_folder, filename)
        
        # Open the TIFF to get its bounds and convert them to a geometry
        with rasterio.open(tif_path) as src:
            tif_geom = box(*src.bounds)
            tif_gdf = gpd.GeoDataFrame(geometry=[tif_geom], crs=src.crs)
        
        # Reproject TIFF geometry to match the CRS of the GeoJSON if needed
        tif_gdf = tif_gdf.to_crs(area_gdf.crs)
        
        # Check if the TIFF's geometry intersects with the GeoJSON's geometry
        if not tif_gdf.intersects(area_gdf.unary_union).any():
            # Move the TIFF to the "out" folder if no intersection
            shutil.move(tif_path, os.path.join(out_folder, filename))

print("Moved all non-intersecting TIFFs to the 'out' folder.")


Moved all non-intersecting TIFFs to the 'out' folder.


In [43]:
import geopandas as gpd
import rasterio
from rasterio.plot import show
from ipyleaflet import Map, GeoJSON, WidgetControl
from ipywidgets import Output, VBox
from glob import glob
import os
from shapely.geometry import box

def get_geometries(tif_files):
    """Extract geometries from TIFF files, create GeoDataFrame, and reproject to EPSG:4326."""
    geometries = []
    for file in tif_files:
        with rasterio.open(file) as src:
            geom = box(*src.bounds)
            geometries.append(geom)
    gdf = gpd.GeoDataFrame(geometry=geometries, crs=src.crs)  # Use the CRS from the TIFF file
    gdf = gdf.to_crs(epsg=4326)  # Reproject to lat/long
    return gdf

def on_click(feature, **kwargs):
    """Handle click events and display TIFF overview."""
    file_path = feature['properties']['path']
    with rasterio.open(file_path) as src:
        out.clear_output()
        with out:
            show(src, 1)

def display_map(geoms):
    """Display geometries on a map with clickable functionality."""
    m = Map(center=(-8.65, 115.2167), zoom=10)  # Centered on Bali
    out = Output()
    
    for index, geom in geoms.iterrows():
        geo_json = GeoJSON(data={"type": "Feature",
                                 "properties": {"path": tif_files[index]},
                                 "geometry": geom.geometry.__geo_interface__},
                           style={"color": "red", "opacity": 0.5, "weight": 1.5, "fillOpacity": 0.5})
        geo_json.on_click(on_click)
        m.add_layer(geo_json)
    
    control = WidgetControl(widget=VBox([out]), position='topright')
    m.add_control(control)
    return m
# List of TIFF files
tif_files = glob(os.path.join('../data/Bali', '*.tif'))
print(f"Found {len(tif_files)} TIFF files." )

# Extract geometries and display map
geoms = get_geometries(tif_files)
display_map(geoms)


Found 341 TIFF files.


Map(center=[-8.65, 115.2167], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…